In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import random

np.set_printoptions(precision=4)

### Loading Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create a dictionary in which keys are all the artists and values are all of the images by that artist:

In [3]:

artistdict = {}

with open('/content/drive/My Drive/train_info.csv', 'r') as csvfile:
  table = csv.reader(csvfile)
  c = 0
  for row in table:
    if c == 0:
      c += 1
      continue
    if row[1] not in artistdict:
      artistdict[row[1]] = []
    artistdict[row[1]].append(row[0])
    c += 1
    
print(c)


79434


Create a balanced set of labeled image pairs:

In [4]:
datapoints = []
badcount=0
for artist in artistdict.keys():
  paintings = artistdict[artist]
  if len(paintings) < 31:
    continue
  for i in range(30):
    left = paintings[i]
    if left[0] != '9':
      continue
    for j in range(i+1, 30):
      right = paintings[j]
      if right[0] != '9':
        continue
      datapoints.append([left, right, 1])
      while True:
        otherartist = random.choice(list(artistdict))
        if otherartist == artist:
          continue
        otherpainting = random.choice(artistdict[otherartist])
        if otherpainting[0] != '9':
          continue
        break

      datapoints.append([left, otherpainting, 0])

random.shuffle(datapoints)

print(len(datapoints))
print(badcount)


6708
0


Process the datapoints into numpy array format:

In [10]:
def names_to_images(datapoint):
  leftimage = datapoint[0]
  rightimage = datapoint[1]
  label = datapoint[2]

  leftfilename = '/content/drive/My Drive/train_9/' + leftimage
  rightfilename = '/content/drive/My Drive/train_9/' + rightimage

  leftimage = tf.io.read_file(leftfilename)
  leftimage = tf.image.decode_jpeg(leftimage, try_recover_truncated=True)
  leftimage = tf.image.convert_image_dtype(leftimage, tf.float32)
  leftimage = tf.image.resize(leftimage, [224, 224])

  rightimage = tf.io.read_file(rightfilename)
  rightimage = tf.image.decode_jpeg(rightimage, try_recover_truncated=True)
  rightimage = tf.image.convert_image_dtype(rightimage, tf.float32)
  rightimage = tf.image.resize(rightimage, [224, 224])


  return leftimage, rightimage, label

m = len(datapoints)
leftimages = np.empty((m, 224, 224, 3))
rightimages = np.empty((m, 224, 224, 3))
labels = np.empty((m))

i=0
print(len(datapoints))

dataset_size = 1470

for datapoint in datapoints:
  leftimage, rightimage, label = names_to_images(datapoint)
  if leftimage.shape[2] != 3 or rightimage.shape[2] != 3:
    continue
  leftimages[i,:,:,:] = leftimage[:,:,:]
  rightimages[i,:,:,:] = rightimage[:,:,:]
  labels[i] = label
  i+=1
  if i == dataset_size:
    break


#img_folder = pathlib.Path('/content/drive/My Drive/train_9')
#image_count = len(list(img_folder.glob('*.jpg')))
#print(image_count)

6708


Define train, val, and test sets:

In [11]:

#dataset = tf.data.Dataset.from_tensor_slices((leftimages[:dataset_size,:,:,:], rightimages[:dataset_size,:,:,:], labels[:dataset_size]))

train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = int(0.15 * dataset_size)
#print(dataset)

#train = dataset.take(train_size)
#val = dataset.skip(train_size).take(val_size)
#test = dataset.skip(train_size).skip(val_size)

leftimg_train = leftimages[:train_size,:,:,:]
leftimg_val = leftimages[train_size:train_size + val_size,:,:,:]
leftimg_test = leftimages[train_size + val_size:,:,:,:]

rightimg_train = rightimages[:train_size,:,:,:]
rightimg_val = rightimages[train_size:train_size + val_size,:,:,:]
rightimg_test = rightimages[train_size + val_size:,:,:,:]

label_train = labels[:train_size]
label_val = labels[train_size:train_size + val_size]
label_test = labels[train_size + val_size:]



print(leftimg_train.shape)

(1029, 224, 224, 3)


Creating Model

In [12]:
input_shape = [224, 224, 3]
left_input = tf.keras.Input(input_shape)
right_input = tf.keras.Input(input_shape)

restransfer = tf.keras.Sequential([
                             hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/feature_vector/1", trainable=False),
                             tf.keras.layers.Dense(800, activation="relu", trainable=True),
                             tf.keras.layers.Dense(128, activation="relu", trainable=True)

]) # inputs size (None, 224, 224, 3) ; outputs size (128)

encoded_l = restransfer(left_input)
encoded_r = restransfer(right_input)

L1_layer = tf.keras.layers.Lambda(lambda tensors:tf.math.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([encoded_l, encoded_r])

prediction = tf.keras.layers.Dense(1, activation="sigmoid", trainable=True)(L1_distance)
siamese_net = tf.keras.Model(inputs=[left_input, right_input], outputs=prediction)

Training the model

In [13]:
siamese_net.compile(loss="binary_crossentropy", metrics=['acc'], optimizer='adam')
siamese_net.summary()
history = siamese_net.fit([leftimg_train, rightimg_train], label_train, epochs=8)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 128)          25302880    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128)          0           sequential_1[0][0]    

Evaluating the model

In [14]:
siamese_net.evaluate([leftimg_val, rightimg_val], label_val)

7/7 [==============================] - 2s 306ms/step - loss: 0.7986 - acc: 0.7045


[0.7985625267028809, 0.7045454382896423]